In [13]:

import sys
sys.path.append('fDNC_Neuron_ID/src')

from scipy.optimize import linear_sum_assignment
import torch
import argparse
from torch.utils.data import DataLoader
import numpy as np
from model import NIT_Registration, neuron_data_pytorch
import pickle
import os

from cpd_p import cpd_p_sep, cpd_p
from cpd_p_color import cpd_p_color
from cpd_rigid_sep import register_rigid
from cpd_nonrigid_sep import register_nonrigid
import matplotlib.pyplot as plt
import time
import scipy.io as sio

import pandas as pd

import warnings

In [3]:
def find_match(label_ref, label_mov):
    if len(label_ref) == 0 or len(label_mov) == 0:
        return []

    pt1_dict = dict()
    labelmov_indices = np.where(label_mov != '')[0]
    for idx1 in labelmov_indices:
        pt1_dict[label_mov[idx1]] = idx1
        
    match = list()
    unlabel = list()
    for idx2, _ in enumerate(label_ref):
        if label_ref[idx2] in pt1_dict:
            match.append([idx2, pt1_dict[label_ref[idx2]]])

        else:
            unlabel.append(idx2)
    
    return np.array(match), np.array(unlabel)

In [14]:
#edited slightly from fDNC base repository code

def register_rigid(x, y, w, max_it=250, x_c=[], y_c=[], fix_scale=False, sigma2c=0.1):
    """
    This function only trannsform x and y coordinate.
    Registers Y to X using the Coherent Point Drift algorithm, in rigid fashion.
    Note: For affine transformation, t = scale*y*r'+1*t'(* is dot). r is orthogonal rotation matrix here.
    Parameters
    ----------
    x : ndarray
        The static shape that y will be registered to. Expected array shape is [n_points_x, n_dims]
    y : ndarray
        The moving shape. Expected array shape is [n_points_y, n_dims]. Note that n_dims should be equal for x and y,
        but n_points does not need to match.
    w : float
        Weight for the outlier suppression. Value is expected to be in range [0.0, 1.0].
    max_it : int
        Maximum number of iterations. The default value is 150.

    Returns
    -------
    t : ndarray
        The transformed version of y. Output shape is [n_points_y, n_dims].
    """
    # get dataset lengths and dimensions
    assert len(x) > 0
    assert len(y) > 0
    [n, d] = x.shape
    [m, d] = y.shape
    assert d >= 2

    if isinstance(x_c, list):
        if len(x_c) and len(y_c):
            use_color = 1
        else:
            use_color = 0
    else:
        if x_c.shape[1] and y_c.shape[1]:
            use_color = 1
        else:
            use_color = 0

    if use_color:
        [nc, dc] = x_c.shape
        [mc, dc] = y_c.shape
    #      sigma2c = (mc*np.trace(np.dot(np.transpose(x_c), x_c))+ nc*np.trace(np.dot(np.transpose(y_c), y_c)) -
    #              2*np.dot(sum(x_c), np.transpose(sum(y_c))))/(mc*nc*dc)
    # t is the updated moving shape,we initialize it with y first.
    t = np.copy(y)
    # initialize sigma^2
    sigma2 = (m * np.trace(np.dot(np.transpose(x), x)) + n * np.trace(np.dot(np.transpose(y), y)) -
              2 * np.dot(sum(x), np.transpose(sum(y)))) / (m * n * d)
    iter = 0
    while (iter < max_it) and (sigma2 > 10.e-8):
        # E step (P matrix)
        if use_color:
            [p1, pt1, px, px_c] = cpd_p_color(x, t, sigma2, w, m, n, d, x_c, y_c, sigma2c)
        else:
            [p1, pt1, p] = cpd_p_sep(x, t, sigma2, w, m, n, d)
        # precompute
        Np = np.sum(pt1)
        Np_max = np.max(p, axis=1)

        # M step, update transformation.
        x_2d, y_2d = x[:, :2], y[:, :2]
        x_z, y_z = x[:, 2:3], y[:, 2:3]
        d = 2
        # align z
        mu_z1 = np.dot(np.transpose(x_z), pt1) / Np
        mu_z2 = np.dot(np.transpose(y_z), p1) / Np
        t_z = mu_z1 - mu_z2
        t[:, 2:3] = y_z + t_z
        # align x and y
        mu_x = np.dot(np.transpose(x_2d), pt1) / Np
        mu_y = np.dot(np.transpose(y_2d), p1) / Np
        # solve for Rotation, scaling, translation and sigma^2
        px = np.dot(p, x_2d)
        a = np.dot(np.transpose(px), y_2d) - Np * (np.dot(mu_x, np.transpose(mu_y)))
        [u, s, v] = np.linalg.svd(a)
        s = np.diag(s)
        c = np.eye(d)
        c[-1, -1] = np.linalg.det(np.dot(u, v))
        r = np.dot(u, np.dot(c, v))
        if fix_scale:
            scale = 1
        else:
            scale = np.trace(np.dot(s, c)) / (sum(sum(y_2d * y_2d * np.matlib.repmat(p1, 1, d))) - Np *
                                              np.dot(np.transpose(mu_y), mu_y))
        sigma22 = np.abs(sum(sum(x_2d * x_2d * np.matlib.repmat(pt1, 1, d))) - Np *
                         np.dot(np.transpose(mu_x), mu_x) - scale * np.trace(np.dot(s, c))) / (Np * d)
        sigma2 = sigma22[0][0]
        #        if use_color:
        #          sigma2c = np.abs((np.sum(x_c*x_c*np.matlib.repmat(pt1, 1, dc))+np.sum(y_c*y_c*np.matlib.repmat(p1, 1, dc)) -
        #                         2*np.trace(np.dot(px_c.T, y_c)))/(Np*dc))
        # ts is translation
        ts = mu_x - np.dot(scale * r, mu_y)
        t_2d = np.dot(scale * y_2d, np.transpose(r)) + np.matlib.repmat(np.transpose(ts), m, 1)
        t[:, :2] = t_2d
        iter = iter + 1
    # if use_color:
    # print('sigma c:{}'.format(sigma2c))
    if x.shape[1] == 3:
        r_out = np.eye(3)
        r_out[:2, :2] = r
        ts_out = np.zeros((3, 1))
        ts_out[:2, :] = ts
        ts_out[2, 0] = t_z
    else:
        r_out = r
        ts_out = ts


    return t, r_out, ts_out, sigma2, np.average(Np_max)


In [15]:
def find_match_dis(mov, ref, match_dict):
    dis_m = np.sum((mov[:, np.newaxis, :] - ref[np.newaxis, :, :]) ** 2, axis=2)
    dis_list = dis_m[match_dict[:, 1], match_dict[:, 0]]
    return dis_list


def cpd_match(mov, ref, mov_col, ref_col, match_dict, best_Np, method='max', plot=False):
    w = 0.1
    lamb = 4e3
    #lamb = 2e3
    beta = 0.25
    # cpd transform
    mov_rigid, _, _, sigma2, Np = register_rigid(ref, mov, w=w, fix_scale=True)
    ori_dis = find_match_dis(mov_rigid, ref, match_dict)
    mov_rigid_inv = np.copy(mov_rigid)
    mov_rigid_inv[:, :2] *= -1
    mov_rigid_inv, _, _, sigma2, Np_inv = register_rigid(ref, mov_rigid_inv, w=w, fix_scale=True)
    inv_dis = find_match_dis(mov_rigid_inv, ref, match_dict)
    if np.mean(ori_dis) > np.mean(inv_dis):
        mov_new = mov_rigid_inv
        Np = Np_inv
    else:
        mov_new = mov_rigid

    if best_Np > Np: #do not run nonrigid if rigid alignment has cost more than 10% lower than the current best for this test file
        return [], None

    mov_nonrigid = register_nonrigid(ref, mov_new, w=w, lamb=lamb, beta=beta)
    # plot the results.
    if plot:
        plt.scatter(mov[:, 0], mov[:, 1], c='red')
        plt.scatter(mov_new[:, 0], mov_new[:, 1], c='yellow')
        plt.scatter(mov_nonrigid[:, 0], mov_nonrigid[:, 1], c='green')
        plt.scatter(ref[:, 0], ref[:, 1], c='black')
        plt.show()

    mov_poscol = np.hstack((mov_nonrigid, mov_col))
    ref_poscol = np.hstack((ref, ref_col))

    #pos_dis = find_match_dis(mov, ref, match_dict)
    #col_dis = find_match_dis(mov_col, ref_col, match_dict)

    if method == 'rigid':
        dis_m = np.sum((mov_new[:, np.newaxis, :] - ref[np.newaxis, :, :]) ** 2, axis=2)
    elif method == 'color':
        dis_m = np.sum((mov_poscol[:, np.newaxis,:]-ref_poscol[np.newaxis,:,:])**2, axis=2)
    else:
        dis_m = np.sum((mov_nonrigid[:, np.newaxis, :] - ref[np.newaxis, :, :]) ** 2, axis=2)

    assigns = {}

    for k in range(5):

        row_inds, col_inds = linear_sum_assignment(dis_m)
        
        assigns[k] = [row_inds, col_inds]

        dis_m[row_inds, col_inds] = np.inf

    return assigns, Np

In [16]:
#Skip files that are too bent or have other issues
skip_files = ['20231013-9-30-0', '20230412-20-15-17', '2023-01-23-01', '20239828-11-14-0', '2023-01-05-01', '2023-01-10-14', '2022-06-28-07', '2022-07-26-01', '2023-01-19-15', '2022-07-15-06', '2022-08-02-01', '2023-01-09-08', '2023-01-09-28', '2023-01-10-14', '2023-01-17-14', '2023-01-19-22', '2023-01-23-01']

In [17]:
aligned_path = 'Data/aligned_heads/aligned_full'

acc = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc2 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc3 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc4 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc5 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))

files = [f for f in os.listdir(aligned_path) if not f.startswith('.') and not f[:-4] in skip_files]
print(len(files))

best_score_idxs = np.zeros(len(files))
best_scores = np.zeros(len(files))
best_assigns = 

with warnings.catch_warnings(action="ignore"):
    for j, test_file in enumerate(files):
        print(j)
        print(test_file)
        curr_best_score = 0
        curr_best_idx = None
        test = pd.read_csv(os.path.join(aligned_path, test_file))
        test = test.drop_duplicates(subset=['ID'], keep='first')
        test = test.fillna('')
        test_gt = np.asarray(test['ID'])
        test_pos = np.asarray(test[['aligned_x', 'aligned_y', 'aligned_z']])
        test_pos = (test_pos - np.median(test_pos, axis=0))/84
        test_pos_aff = np.asarray(test[['real_X', 'real_Y', 'real_Z']])
        test_pos_aff = (test_pos_aff - np.median(test_pos_aff, axis=0))/84
        test_col = np.asarray(test[['real_R', 'real_G', 'real_B']])
        test_col = ((test_col - np.min(test_col, axis=0))/(np.max(test_col, axis=0)-np.min(test_col, axis=0))-0.5)/5

        num_labels = len(test[test['ID']!=''])
        print(num_labels)

        for i, temp_file in enumerate(files):
            if j == i:
                continue
            temp = pd.read_csv(os.path.join(aligned_path, temp_file))
            temp = temp.drop_duplicates(subset=['ID'], keep='first' )
            temp = temp.fillna('')
            temp_gt = np.asarray(temp['ID'])
            temp_pos = np.asarray(temp[['aligned_x', 'aligned_y', 'aligned_z']])
            temp_pos = (temp_pos - np.median(temp_pos, axis=0))/84
            temp_pos_aff = np.asarray(temp[['real_X', 'real_Y', 'real_Z']])
            temp_pos_aff = (temp_pos_aff - np.median(temp_pos_aff, axis=0))/84
            temp_col = np.asarray(temp[['real_R', 'real_G', 'real_B']])
            temp_col = ((temp_col - np.min(temp_col, axis=0))/(np.max(temp_col, axis=0)-np.min(temp_col, axis=0))-0.5)/5

            match, unlabel = find_match(temp_gt, test_gt)

            assigns, score = cpd_match(test_pos, temp_pos, test_col, temp_col,match, curr_best_score, method='color',plot=False)
            corr = np.zeros((len(test_gt),5))

            if score is None:
                continue
            elif score > curr_best_score:
                curr_best_score = score
                curr_best_idx = i

                for k in range(5):
                    row, col = assigns[k]

                    for l, idx in enumerate(row):
                        col_label = col[l]

                        match_idx = np.argwhere(match[:,1]==idx)
                        match_ref = match[match_idx, 0]

                        if col_label == match_ref:
                            corr[idx, k] =1

                corr1 = corr[:,0]
                corr2 = np.logical_or(corr[:,0], corr[:,1])
                corr3 = np.logical_or(corr2, corr[:,2])
                corr4 = np.logical_or(corr3, corr[:,3])
                corr5 = np.logical_or(corr4, corr[:,4])
                
                acc[j, i] =  np.sum(corr1)/num_labels
                acc2[j, i] =  np.sum(corr2)/num_labels
                acc3[j, i] =  np.sum(corr3)/num_labels 
                acc4[j, i] =  np.sum(corr4)/num_labels
                acc5[j, i] =  np.sum(corr5)/num_labels

        best_scores[j] = curr_best_score
        best_score_idxs[j] = curr_best_idx


best_temp = {}
CPD_df = pd.DataFrame(columns=['Filename', 'Best_template', 'top1', 'top2','top3', 'top4', 'top5'])

for i, file in enumerate(files):
    best_idx = int(best_score_idxs[i])
    best_file = files[best_idx]
    best_temp[file] = (best_file, acc[i,best_idx], acc5[i,best_idx])
                       
    CPD_df.loc[len(CPD_df)] = [file[:-4], best_file[:-4],acc[i,best_idx], acc2[i,best_idx], acc3[i,best_idx], acc4[i,best_idx] ,acc5[i,best_idx]]

CPD_df.to_csv('/Users/danielysprague/foco_lab/data/Acc_CPD/match_all_temp.csv')

104
0
6.csv
67
1
7.csv
65
2
2022-12-21-06.csv
54
3
20221106-21-00-09.csv
33
4
20230322-20-16-50.csv
44
5
5.csv
64
6
2023-01-06-08.csv
72
7
4.csv
63
8
20230918-17-21-0.csv
156
9
2021-12-03-w00-NP1.csv
57
10
20221106-21-47-31.csv
53
11
20230322-22-43-03.csv
42
12
20230506-12-56-00.csv
47
13
20230506-14-24-57.csv
51
14
1.csv
62
15
38_YAaDV.csv
189
16
2023-01-16-08.csv
68
17
20230510-13-25-46.csv
46
18
56_YAaDV.csv
190
19
20190928_08.csv
183
20
3.csv
64
21
2023-01-16-22.csv
78
22
2.csv
58
23
2023-01-13-07.csv
72
24
20191104_10.csv
174
25
20190929_06.csv
175
26
20230928-14-27-0.csv
154
27
64_YAaDV.csv
189
28
2023-01-23-15.csv
78
29
20190929_07.csv
176
30
2023-01-17-07.csv
79
31
20191030_03.csv
182
32
20190929_05.csv
168
33
2022-03-05-w00-NP1.csv
52
34
20190925_04.csv
176
35
11_YAaLR.csv
193
36
76_YAaDV.csv
188
37
2022-07-15-12.csv
64
38
20191030_07.csv
175
39
20190925_01.csv
180
40
2023-01-17-01.csv
83
41
20230928-11-14-0.csv
157
42
2023-01-09-15.csv
95
43
20190929_03.csv
175
44
20231013-11

In [18]:
aligned_path = 'Data/aligned_heads/aligned_NP_nomatch'


acc = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc2 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc3 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc4 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc5 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))

test_files = [f for f in os.listdir(aligned_path) if not f.startswith('.') and not f[:-4] in skip_files]
temp_files = [f for f in os.listdir(aligned_path) if f[:-4]+'.nwb' in os.listdir('/Users/danielysprague/foco_lab/data/final_nwb/NP_og')]
print(len(files))

best_score_idxs = np.zeros(len(files))
best_scores = np.zeros(len(files))

with warnings.catch_warnings(action="ignore"):
    for j, test_file in enumerate(test_files):
        print(j)
        print(test_file)
        curr_best_score = 0
        curr_best_idx = None
        test = pd.read_csv(os.path.join(aligned_path, test_file))
        test = test.drop_duplicates(subset=['ID'], keep='first')
        test = test.fillna('')
        test_gt = np.asarray(test['ID'])
        test_pos = np.asarray(test[['aligned_x', 'aligned_y', 'aligned_z']])
        test_pos = (test_pos - np.median(test_pos, axis=0))/84
        test_pos_aff = np.asarray(test[['real_X', 'real_Y', 'real_Z']])
        test_pos_aff = (test_pos_aff - np.median(test_pos_aff, axis=0))/84
        test_col = np.asarray(test[['real_R', 'real_G', 'real_B']])
        test_col = ((test_col - np.min(test_col, axis=0))/(np.max(test_col, axis=0)-np.min(test_col, axis=0))-0.5)/5

        num_labels = len(test[test['ID']!=''])
        print(num_labels)

        for i, temp_file in enumerate(temp_files):
            if j == i:
                continue
            temp = pd.read_csv(os.path.join(aligned_path, temp_file))
            temp = temp.drop_duplicates(subset=['ID'], keep='first' )
            temp = temp.fillna('')
            temp_gt = np.asarray(temp['ID'])
            temp_pos = np.asarray(temp[['aligned_x', 'aligned_y', 'aligned_z']])
            temp_pos = (temp_pos - np.median(temp_pos, axis=0))/84
            temp_pos_aff = np.asarray(temp[['real_X', 'real_Y', 'real_Z']])
            temp_pos_aff = (temp_pos_aff - np.median(temp_pos_aff, axis=0))/84
            temp_col = np.asarray(temp[['real_R', 'real_G', 'real_B']])
            temp_col = ((temp_col - np.min(temp_col, axis=0))/(np.max(temp_col, axis=0)-np.min(temp_col, axis=0))-0.5)/5

            match, unlabel = find_match(temp_gt, test_gt)

            assigns, score = cpd_match(test_pos, temp_pos, test_col, temp_col,match, curr_best_score, method='color',plot=False)
            corr = np.zeros((len(test_gt),5))

            if score is None:
                continue
            elif score > curr_best_score:
                curr_best_score = score
                curr_best_idx = i
                curr_best_assign = assigns

                for k in range(5):
                    row, col = assigns[k]

                    for l, idx in enumerate(row):
                        col_label = col[l]

                        match_idx = np.argwhere(match[:,1]==idx)
                        match_ref = match[match_idx, 0]

                        if col_label == match_ref:
                            corr[idx, k] =1

                corr1 = corr[:,0]
                corr2 = np.logical_or(corr[:,0], corr[:,1])
                corr3 = np.logical_or(corr2, corr[:,2])
                corr4 = np.logical_or(corr3, corr[:,3])
                corr5 = np.logical_or(corr4, corr[:,4])
                
                acc[j, i] =  np.sum(corr1)/num_labels
                acc2[j, i] =  np.sum(corr2)/num_labels
                acc3[j, i] =  np.sum(corr3)/num_labels 
                acc4[j, i] =  np.sum(corr4)/num_labels
                acc5[j, i] =  np.sum(corr5)/num_labels

        best_scores[j] = curr_best_score
        best_score_idxs[j] = curr_best_idx

best_temp = {}
CPD_df = pd.DataFrame(columns=['Filename', 'Best_template', 'top1', 'top2','top3', 'top4', 'top5'])

for i, file in enumerate(test_files):
    best_idx = int(best_score_idxs[i])
    best_file = temp_files[best_idx]
    best_temp[file] = (best_file, acc[i,best_idx], acc5[i,best_idx])
                       
    CPD_df.loc[len(CPD_df)] = [file[:-4], best_file[:-4],acc[i,best_idx], acc2[i,best_idx], acc3[i,best_idx], acc4[i,best_idx] ,acc5[i,best_idx]]

CPD_df.to_csv('Data/Acc_CPD/nomatch_NP_temp.csv')

104
0
6.csv
67
1
7.csv
65
2
2022-12-21-06.csv
54
3
20221106-21-00-09.csv
33
4
20230322-20-16-50.csv
44
5
5.csv
64
6
2023-01-06-08.csv
72
7
4.csv
63
8
20230918-17-21-0.csv
156
9
2021-12-03-w00-NP1.csv
57
10
20221106-21-47-31.csv
53
11
20230322-22-43-03.csv
42
12
20230506-12-56-00.csv
47
13
20230506-14-24-57.csv
51
14
1.csv
62
15
38_YAaDV.csv
189
16
2023-01-16-08.csv
68
17
20230510-13-25-46.csv
46
18
56_YAaDV.csv
190
19
20190928_08.csv
183
20
3.csv
64
21
2023-01-16-22.csv
78
22
2.csv
58
23
2023-01-13-07.csv
72
24
20191104_10.csv
174
25
20190929_06.csv
175
26
20230928-14-27-0.csv
154
27
64_YAaDV.csv
189
28
2023-01-23-15.csv
78
29
20190929_07.csv
176
30
2023-01-17-07.csv
79
31
20191030_03.csv
182
32
20190929_05.csv
168
33
2022-03-05-w00-NP1.csv
52
34
20190925_04.csv
176
35
11_YAaLR.csv
193
36
76_YAaDV.csv
188
37
2022-07-15-12.csv
64
38
20191030_07.csv
175
39
20190925_01.csv
180
40
2023-01-17-01.csv
83
41
20230928-11-14-0.csv
157
42
2023-01-09-15.csv
95
43
20190929_03.csv
175
44
20231013-11

In [33]:
aligned_path = 'Data/aligned_heads/aligned_NP'


acc = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc2 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc3 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc4 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))
acc5 = np.zeros((len(os.listdir(aligned_path)), len(os.listdir(aligned_path))))

test_files = [f for f in os.listdir(aligned_path) if not f.startswith('.') and not f[:-4] in skip_files]
temp_files = [f for f in os.listdir(aligned_path) if f[:-4]+'.nwb' in os.listdir('/Users/danielysprague/foco_lab/data/final_nwb/NP_og')]
print(len(files))

best_score_idxs = np.zeros(len(files))
best_scores = np.zeros(len(files))

with warnings.catch_warnings(action="ignore"):
    for j, test_file in enumerate(test_files):
        print(j)
        print(test_file)
        curr_best_score = 0
        curr_best_idx = None
        test = pd.read_csv(os.path.join(aligned_path, test_file))
        test = test.drop_duplicates(subset=['ID'], keep='first')
        test = test.fillna('')
        test_gt = np.asarray(test['ID'])
        test_pos = np.asarray(test[['aligned_x', 'aligned_y', 'aligned_z']])
        test_pos = (test_pos - np.median(test_pos, axis=0))/84
        test_pos_aff = np.asarray(test[['real_X', 'real_Y', 'real_Z']])
        test_pos_aff = (test_pos_aff - np.median(test_pos_aff, axis=0))/84
        test_col = np.asarray(test[['real_R', 'real_G', 'real_B']])
        test_col = ((test_col - np.min(test_col, axis=0))/(np.max(test_col, axis=0)-np.min(test_col, axis=0))-0.5)/5

        num_labels = len(test[test['ID']!=''])
        print(num_labels)

        for i, temp_file in enumerate(temp_files):
            if j == i:
                continue
            temp = pd.read_csv(os.path.join(aligned_path, temp_file))
            temp = temp.drop_duplicates(subset=['ID'], keep='first' )
            temp = temp.fillna('')
            temp_gt = np.asarray(temp['ID'])
            temp_pos = np.asarray(temp[['aligned_x', 'aligned_y', 'aligned_z']])
            temp_pos = (temp_pos - np.median(temp_pos, axis=0))/84
            temp_pos_aff = np.asarray(temp[['real_X', 'real_Y', 'real_Z']])
            temp_pos_aff = (temp_pos_aff - np.median(temp_pos_aff, axis=0))/84
            temp_col = np.asarray(temp[['real_R', 'real_G', 'real_B']])
            temp_col = ((temp_col - np.min(temp_col, axis=0))/(np.max(temp_col, axis=0)-np.min(temp_col, axis=0))-0.5)/5

            match, unlabel = find_match(temp_gt, test_gt)

            assigns, score = cpd_match(test_pos, temp_pos, test_col, temp_col,match, curr_best_score, method='color',plot=False)
            corr = np.zeros((len(test_gt),5))

            if score is None:
                continue
            elif score > curr_best_score:
                curr_best_score = score
                curr_best_idx = i

                for k in range(5):
                    row, col = assigns[k]

                    for l, idx in enumerate(row):
                        col_label = col[l]

                        match_idx = np.argwhere(match[:,1]==idx)
                        match_ref = match[match_idx, 0]

                        if col_label == match_ref:
                            corr[idx, k] =1

                corr1 = corr[:,0]
                corr2 = np.logical_or(corr[:,0], corr[:,1])
                corr3 = np.logical_or(corr2, corr[:,2])
                corr4 = np.logical_or(corr3, corr[:,3])
                corr5 = np.logical_or(corr4, corr[:,4])
                
                acc[j, i] =  np.sum(corr1)/num_labels
                acc2[j, i] =  np.sum(corr2)/num_labels
                acc3[j, i] =  np.sum(corr3)/num_labels 
                acc4[j, i] =  np.sum(corr4)/num_labels
                acc5[j, i] =  np.sum(corr5)/num_labels

        best_scores[j] = curr_best_score
        best_score_idxs[j] = curr_best_idx

best_temp = {}
CPD_df = pd.DataFrame(columns=['Filename', 'Best_template', 'top1', 'top2','top3', 'top4', 'top5'])

for i, file in enumerate(test_files):
    best_idx = int(best_score_idxs[i])
    best_file = temp_files[best_idx]
    best_temp[file] = (best_file, acc[i,best_idx], acc5[i,best_idx])
                       
    CPD_df.loc[len(CPD_df)] = [file[:-4], best_file[:-4],acc[i,best_idx], acc2[i,best_idx], acc3[i,best_idx], acc4[i,best_idx] ,acc5[i,best_idx]]

CPD_df.to_csv('Data/Acc_CPD/match_NP_temp.csv')

104
0
6.csv
67


1
7.csv
65
2
2022-12-21-06.csv
54
3
20221106-21-00-09.csv
33
4
20230322-20-16-50.csv
44
5
5.csv
64
6
2023-01-06-08.csv
72
7
4.csv
63
8
20230918-17-21-0.csv
156
9
2021-12-03-w00-NP1.csv
57
10
20221106-21-47-31.csv
53
11
20230322-22-43-03.csv
42
12
20230506-12-56-00.csv
47
13
20230506-14-24-57.csv
51
14
1.csv
62
15
38_YAaDV.csv
189
16
2023-01-16-08.csv
68
17
20230510-13-25-46.csv
46
18
56_YAaDV.csv
190
19
20190928_08.csv
183
20
3.csv
64
21
2023-01-16-22.csv
78
22
2.csv
58
23
2023-01-13-07.csv
72
24
20191104_10.csv
174
25
20190929_06.csv
175
26
20230928-14-27-0.csv
154
27
64_YAaDV.csv
189
28
2023-01-23-15.csv
78
29
20190929_07.csv
176
30
2023-01-17-07.csv
79
31
20191030_03.csv
182
32
20190929_05.csv
168
33
2022-03-05-w00-NP1.csv
52
34
20190925_04.csv
176
35
11_YAaLR.csv
193
36
76_YAaDV.csv
188
37
2022-07-15-12.csv
64
38
20191030_07.csv
175
39
20190925_01.csv
180
40
2023-01-17-01.csv
83
41
20230928-11-14-0.csv
157
42
2023-01-09-15.csv
95
43
20190929_03.csv
175
44
20231013-11-11-0.csv
163
4

# 